In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton

In [ ]:
import pandas as pd
import json
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Limpeza e pré-processamento

In [ ]:
data = pd.read_json("/content/drive/MyDrive/FineTuning/trn_100k.json", lines=True)

In [ ]:
data.drop(columns=['uid', 'target_ind', 'target_rel'], axis=1, inplace=True)

In [ ]:
data = data.drop(data[data["content"] == ""].index)

In [ ]:
data

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
3,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
12,The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.
...,...,...
99994,Unmasking Europa: The Search for Life on Jupit...,"From the reviews:""Unmasking Europa &#x2026; br..."
99995,PET: Molecular Imaging and Its Biological Appl...,From the reviews of the first edition:RAD Maga...
99996,Pathology of Pediatric Gastrointestinal and Li...,Pediatric gastrointestinal and liver biopsies ...
99997,Lithium-Ion Batteries: Science and Technologies,"In developing electrochemical cells, one must ..."


# Fine Tuning

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 2412,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "GIVE ME MORE INFORMATION ABOUT THIS ITEM",
        "Microbiology: A Photographic Atlas for the Laboratory", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
GIVE ME MORE INFORMATION ABOUT THIS ITEM

### Input:
Microbiology: A Photographic Atlas for the Laboratory

### Response:
This book is a laboratory manual that provides step-by-step instructions for laboratory exercises and includes color photographs of the results of each experiment.

### Score:
This response accurately answers the request for information.

### Instruction:
GIVE ME MORE INFORMATION ABOUT THIS ITEM

### Input:
Microbiology: A Photographic Atlas for the Laboratory

### Response:
This book is a laboratory manual that provides step-by-step instructions for laboratory exercises and includes color photographs of the results of each experiment.

### Score:
This response accurately answers the request for information.

### Instruction:
GIVE ME MORE INFORMATION ABOUT THIS ITEM

### Input

In [ ]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['title', 'content', '__index_level_0__'],
    num_rows: 75064
})

In [ ]:
def generate_text(item):
  for x in item:
    text = alpaca_prompt.format("GIVE ME MORE INFORMATION ABOUT THIS ITEM", item["title"], item["content"]) + EOS_TOKEN
  return { "text": text }


dataset = dataset.map(generate_text)
dataset

Map:   0%|          | 0/75064 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'content', '__index_level_0__', 'text'],
    num_rows: 75064
})

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 2412,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/75064 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 75,064 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.712100
2,2.515500
3,2.449600
4,2.415200
5,2.362200
6,2.461100
7,2.233600
8,2.153800
9,1.974600
10,1.937200


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "GIVE ME MORE INFORMATION ABOUT THIS ITEM",
        "Microbiology: A Photographic Atlas for the Laboratory", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
GIVE ME MORE INFORMATION ABOUT THIS ITEM

### Input:
Microbiology: A Photographic Atlas for the Laboratory

### Response:
Microbiology: A Photographic Atlas for the Laboratory, 5th Edition is an easy-to-use, full-color atlas that provides an excellent visual overview of the major concepts and techniques of microbiology. The text features over 400 photographs, including many new images, which provide a realistic view of the laboratory experience. The new edition features a new full-color design, a new chapter on medical microbiology, and new laboratory exercises. This atlas is ideal for laboratory courses in microbiology or as a reference for microbiology laboratory technicians.<|end_of_text|>
